## 1. 분석목표

In [24]:
import os
import csv
import pandas as pd
import numpy as np

cwd = os.getcwd()
print("current working directory:", cwd)

current working directory: c:\Users\sci\Desktop\rfm_dacon


In [25]:
def read_folder_csv(directory):
    file_list = [name for name in os.listdir(directory) if name.endswith(".csv")] #startwith(), xx in file, xx not in file, endswith()
    print(file_list)
    data_dict = {}
    for i in file_list:
        data_dict[i] = pd.read_csv(os.path.join(directory, i))
    return data_dict
    

def read_folder_json(directory):
    dict_list = []
    for i in file_list:
        for line in open((path+i),"r"):
            dict_list.append(json.loads(line))
    df = pd.DataFrame(dict_list)

In [43]:
root_path = "C:/Users/sci/Desktop/Data/dacon_segment_1/"

data_dict = read_folder_csv(root_path)

['Customer_info.csv', 'Discount_info.csv', 'Marketing_info.csv', 'Onlinesales_info.csv', 'Tax_info.csv']


In [49]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_row', 10)

data_dict

{'Customer_info.csv':            고객ID 성별        고객지역  가입기간
 0     USER_1358  남     Chicago    12
 1     USER_0190  남  California    43
 2     USER_0066  남     Chicago    33
 3     USER_0345  여  California    30
 4     USER_0683  남  California    49
 ...         ... ..         ...   ...
 1463  USER_0513  여    New York    41
 1464  USER_0167  여     Chicago    48
 1465  USER_0845  남  New Jersey    19
 1466  USER_0504  여    New York    45
 1467  USER_0562  여  California     7
 
 [1468 rows x 4 columns],
 'Discount_info.csv':        월                제품카테고리    쿠폰코드  할인율
 0    Jan               Apparel  SALE10   10
 1    Feb               Apparel  SALE20   20
 2    Mar               Apparel  SALE30   30
 3    Jan              Nest-USA  ELEC10   10
 4    Feb              Nest-USA  ELEC20   20
 ..   ...                   ...     ...  ...
 199  Nov  Notebooks & Journals    NJ20   20
 200  Dec  Notebooks & Journals    NJ30   30
 201  Oct               Android   AND10   10
 202  Nov               

- 'Customer_info.csv'와 'Onlinesales_info.csv'의 고객ID
- 'Discount_info.csv'와 'Onlinesales_info.csv'의 제품카테고리+월
- 'Marketing_info.csv' 날짜와 'Onlinesales_info.csv' 거래날짜
- 'Tax_info.csv'와 'Onlinesales_info.csv'의 제품카테고리

In [74]:
# dict에서 각각 파일명 지정
def split_filename(dict):
    for i in dict.keys():
        filenames = os.path.splitext(i)[0]
        globals()[filenames] = dict[i]

In [75]:
split_filename(data_dict)

In [79]:
# print(Customer_info, Discount_info, Marketing_info, Onlinesales_info, Tax_info)

In [102]:
#merge
test = Onlinesales_info.merge(Customer_info, how = 'left', on = '고객ID')

Marketing_info.rename(columns = {'날짜': '거래날짜'}, inplace = True)
test = test.merge(Marketing_info, how = 'left', on = '거래날짜')
test = test.merge(Tax_info, how = 'left', on = '제품카테고리')

In [103]:
test

,고객ID,거래ID,거래날짜,제품ID,제품카테고리,수량,평균금액,배송료,쿠폰상태,성별,고객지역,가입기간,오프라인비용,온라인비용,GST
0,USER_1358,Transaction_0000,2019-01-01,Product_0981,Nest-USA,1,153.71,6.50,Used,남,Chicago,12,4500,2424.50,0.10
1,USER_1358,Transaction_0001,2019-01-01,Product_0981,Nest-USA,1,153.71,6.50,Used,남,Chicago,12,4500,2424.50,0.10
2,USER_1358,Transaction_0002,2019-01-01,Product_0904,Office,1,2.05,6.50,Used,남,Chicago,12,4500,2424.50,0.10
3,USER_1358,Transaction_0003,2019-01-01,Product_0203,Apparel,5,17.53,6.50,Not Used,남,Chicago,12,4500,2424.50,0.18
4,USER_1358,Transaction_0003,2019-01-01,Product_0848,Bags,1,16.50,6.50,Used,남,Chicago,12,4500,2424.50,0.18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52919,USER_0504,Transaction_25056,2019-12-31,Product_0976,Nest-USA,1,121.30,6.50,Clicked,여,New York,45,4000,2058.75,0.10
52920,USER_0504,Transaction_25057,2019-12-31,Product_0413,Apparel,1,48.92,6.50,Used,여,New York,45,4000,2058.75,0.18
52921,USER_0504,Transaction_25058,2019-12-31,Product_0989,Nest-USA,1,151.88,6.50,Used,여,New York,45,4000,2058.75,0.10
52922,USER_0562,Transaction_25059,2019-12-31,Product_0985,Nest-USA,5,80.52,6.50,Clicked,여,California,7,4000,2058.75,0.10


In [104]:
# key 만들기
Discount_info['key'] = Discount_info['월'] + Discount_info['제품카테고리']

In [106]:
from datetime import datetime
import locale

# 현재 locale 저장
old_locale = locale.getlocale(locale.LC_TIME)

# 임시로 locale을 영어로 설정
locale.setlocale(locale.LC_TIME, 'en_US.UTF-8')

#날짜 -> 영어약자로 변경
test['거래날짜'] = pd.to_datetime(test['거래날짜'])
test['월'] = test['거래날짜'].apply(lambda x: x.strftime('%b'))

# locale을 원래대로 복구
locale.setlocale(locale.LC_TIME, old_locale)

'en_US.UTF-8'

In [108]:
test['key'] = test['월'] + test['제품카테고리']
test

,고객ID,거래ID,거래날짜,제품ID,제품카테고리,수량,평균금액,배송료,쿠폰상태,성별,고객지역,가입기간,오프라인비용,온라인비용,GST,월,key
0,USER_1358,Transaction_0000,2019-01-01,Product_0981,Nest-USA,1,153.71,6.50,Used,남,Chicago,12,4500,2424.50,0.10,Jan,JanNest-USA
1,USER_1358,Transaction_0001,2019-01-01,Product_0981,Nest-USA,1,153.71,6.50,Used,남,Chicago,12,4500,2424.50,0.10,Jan,JanNest-USA
2,USER_1358,Transaction_0002,2019-01-01,Product_0904,Office,1,2.05,6.50,Used,남,Chicago,12,4500,2424.50,0.10,Jan,JanOffice
3,USER_1358,Transaction_0003,2019-01-01,Product_0203,Apparel,5,17.53,6.50,Not Used,남,Chicago,12,4500,2424.50,0.18,Jan,JanApparel
4,USER_1358,Transaction_0003,2019-01-01,Product_0848,Bags,1,16.50,6.50,Used,남,Chicago,12,4500,2424.50,0.18,Jan,JanBags
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52919,USER_0504,Transaction_25056,2019-12-31,Product_0976,Nest-USA,1,121.30,6.50,Clicked,여,New York,45,4000,2058.75,0.10,Dec,DecNest-USA
52920,USER_0504,Transaction_25057,2019-12-31,Product_0413,Apparel,1,48.92,6.50,Used,여,New York,45,4000,2058.75,0.18,Dec,DecApparel
52921,USER_0504,Transaction_25058,2019-12-31,Product_0989,Nest-USA,1,151.88,6.50,Used,여,New York,45,4000,2058.75,0.10,Dec,DecNest-USA
52922,USER_0562,Transaction_25059,2019-12-31,Product_0985,Nest-USA,5,80.52,6.50,Clicked,여,California,7,4000,2058.75,0.10,Dec,DecNest-USA


In [110]:
test = test.merge(Discount_info, how = 'left', on = 'key')

In [112]:
test.drop(columns = ['월_x', '월_y', 'key', '제품카테고리_y'], axis=1, inplace=True)
test.rename(columns = {'제품카테고리_x': '제품카테고리'}, inplace = True)

In [113]:
test.head()

,고객ID,거래ID,거래날짜,제품ID,제품카테고리,수량,평균금액,배송료,쿠폰상태,성별,고객지역,가입기간,오프라인비용,온라인비용,GST,쿠폰코드,할인율
0,USER_1358,Transaction_0000,2019-01-01,Product_0981,Nest-USA,1,153.71,6.5,Used,남,Chicago,12,4500,2424.5,0.10,ELEC10,10.0
1,USER_1358,Transaction_0001,2019-01-01,Product_0981,Nest-USA,1,153.71,6.5,Used,남,Chicago,12,4500,2424.5,0.10,ELEC10,10.0
2,USER_1358,Transaction_0002,2019-01-01,Product_0904,Office,1,2.05,6.5,Used,남,Chicago,12,4500,2424.5,0.10,OFF10,10.0
3,USER_1358,Transaction_0003,2019-01-01,Product_0203,Apparel,5,17.53,6.5,Not Used,남,Chicago,12,4500,2424.5,0.18,SALE10,10.0
4,USER_1358,Transaction_0003,2019-01-01,Product_0848,Bags,1,16.50,6.5,Used,남,Chicago,12,4500,2424.5,0.18,AIO10,10.0


In [1]:
test.shape

NameError: name 'test' is not defined